# ETAPA 2: Análise do Perfil dos Respondentes

## Consulta à Comunidade UFPE sobre o Uso de Inteligência Artificial

---

**Objetivo:** Caracterizar a amostra e avaliar sua representatividade.

**Variáveis analisadas:**
- Vínculo com a UFPE (Discente Graduação, Discente Pós, Docente, Técnico, Egresso)
- Centro/Unidade Acadêmica (34 categorias)
- Curso (para discentes)

**Procedimentos:**
1. Calcular distribuição de frequências absolutas e relativas por vínculo
2. Calcular distribuição por Centro/Unidade
3. Gerar tabelas cruzadas (vínculo × centro)
4. Comparar proporções da amostra com dados institucionais da UFPE (se disponíveis)
5. Analisar distribuição temporal das respostas ao longo do período de coleta

**Produto:** Tabelas e gráficos de caracterização da amostra; análise de representatividade.

**Critério de Validação:** Coerência com dados institucionais e cobertura de todas as unidades acadêmicas.

---

## Requisitos Técnicos

### Kernel Python
- **Python:** 3.9 ou superior
- **Kernel recomendado:** `python3` ou ambiente virtual com as dependências instaladas

### Dados de Entrada

| Arquivo | Descrição | Origem |
|---------|-----------|--------|
| `dados_limpos_etapa1.xlsx` | Base de dados limpa com 2.164 registros | Etapa 1 |

### Dados de Saída

| Arquivo | Descrição |
|---------|----------|
| `perfil_respondentes_etapa2.xlsx` | Tabelas de frequência e cruzamentos |
| `graficos_perfil_etapa2.png` | Visualizações do perfil da amostra |
| `relatorio_perfil_etapa2.md` | Relatório descritivo da caracterização |

---

## 1. Instalação de Dependências

In [ ]:
# ============================================================================
# INSTALAÇÃO DE DEPENDÊNCIAS
# Execute esta célula apenas uma vez para instalar os pacotes necessários
# ============================================================================

!pip install pandas openpyxl matplotlib seaborn numpy --quiet

print("✅ Dependências instaladas com sucesso!")

## 2. Configuração Inicial

In [ ]:
# ============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Configurações de visualização
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
sns.set_style("whitegrid")
sns.set_palette("Set2")

# Verificar versões
print("Bibliotecas carregadas com sucesso!")
print(f"  • Pandas: {pd.__version__}")
print(f"  • Matplotlib: {plt.matplotlib.__version__}")
print(f"  • Seaborn: {sns.__version__}")

In [ ]:
# ============================================================================
# CONFIGURAÇÃO DE ARQUIVOS DE ENTRADA E SAÍDA
# ============================================================================

# Arquivo de entrada (gerado na Etapa 1)
ARQUIVO_ENTRADA = "dados_limpos_etapa1.xlsx"

# Arquivos de saída (serão usados nas próximas etapas)
ARQUIVO_SAIDA_EXCEL = "perfil_respondentes_etapa2.xlsx"
ARQUIVO_SAIDA_GRAFICOS = "graficos_perfil_etapa2.png"
ARQUIVO_SAIDA_RELATORIO = "relatorio_perfil_etapa2.md"

print("Configuração de arquivos:")
print(f"  📥 Entrada: {ARQUIVO_ENTRADA}")
print(f"  📤 Saídas:  {ARQUIVO_SAIDA_EXCEL}")
print(f"            {ARQUIVO_SAIDA_GRAFICOS}")
print(f"            {ARQUIVO_SAIDA_RELATORIO}")

## 3. Carregamento dos Dados

In [ ]:
# Carregar dados limpos da Etapa 1
try:
    df = pd.read_excel(ARQUIVO_ENTRADA)
    print(f"✅ Dados carregados com sucesso!")
    print(f"   Registros: {len(df)}")
    print(f"   Variáveis: {len(df.columns)}")
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{ARQUIVO_ENTRADA}' não encontrado!")
    print(f"   Certifique-se de que a Etapa 1 foi executada e o arquivo está no diretório correto.")
    raise

In [ ]:
# Verificar estrutura dos dados
print("Colunas disponíveis:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2}. {col}")

In [ ]:
# Visualizar primeiras linhas
df.head(3)

---

## 4. Análise por Vínculo Institucional

### 4.1 Distribuição de Frequências

In [ ]:
# Calcular frequências absolutas e relativas por vínculo
freq_vinculo = df['Vinculo_Padronizado'].value_counts().reset_index()
freq_vinculo.columns = ['Vínculo', 'Frequência']
freq_vinculo['Percentual'] = (freq_vinculo['Frequência'] / freq_vinculo['Frequência'].sum() * 100).round(1)
freq_vinculo['Percentual_Acum'] = freq_vinculo['Percentual'].cumsum().round(1)

# Adicionar linha de total
total_row = pd.DataFrame([{
    'Vínculo': 'TOTAL',
    'Frequência': freq_vinculo['Frequência'].sum(),
    'Percentual': 100.0,
    'Percentual_Acum': '-'
}])
freq_vinculo_display = pd.concat([freq_vinculo, total_row], ignore_index=True)

print("=" * 60)
print("DISTRIBUIÇÃO POR VÍNCULO INSTITUCIONAL")
print("=" * 60)
print(freq_vinculo_display.to_string(index=False))

In [ ]:
# Gráfico de barras - Vínculo Institucional
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
cores = sns.color_palette("Set2", n_colors=len(freq_vinculo))
bars = axes[0].barh(freq_vinculo['Vínculo'], freq_vinculo['Frequência'], color=cores)
axes[0].set_xlabel('Número de Respondentes')
axes[0].set_title('Distribuição por Vínculo Institucional')
axes[0].invert_yaxis()

# Adicionar labels nas barras
for bar, freq, pct in zip(bars, freq_vinculo['Frequência'], freq_vinculo['Percentual']):
    axes[0].text(bar.get_width() + 10, bar.get_y() + bar.get_height()/2, 
                 f'{freq} ({pct}%)', va='center', fontsize=10)

# Gráfico de pizza
axes[1].pie(freq_vinculo['Frequência'], labels=freq_vinculo['Vínculo'], 
            autopct='%1.1f%%', colors=cores, startangle=90)
axes[1].set_title('Proporção por Vínculo Institucional')

plt.tight_layout()
plt.savefig('grafico_vinculo_etapa2.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_vinculo_etapa2.png")

---

## 5. Análise por Centro/Unidade Acadêmica

### 5.1 Distribuição de Frequências

In [ ]:
# Calcular frequências por Centro/Unidade
freq_centro = df['Centro_Unidade'].value_counts().reset_index()
freq_centro.columns = ['Centro/Unidade', 'Frequência']
freq_centro['Percentual'] = (freq_centro['Frequência'] / freq_centro['Frequência'].sum() * 100).round(1)
freq_centro['Percentual_Acum'] = freq_centro['Percentual'].cumsum().round(1)

print("=" * 80)
print("DISTRIBUIÇÃO POR CENTRO/UNIDADE ACADÊMICA")
print("=" * 80)
print(freq_centro.to_string(index=False))
print(f"\nTotal de Centros/Unidades: {len(freq_centro)}")

In [ ]:
# Gráfico de barras horizontais - Centro/Unidade
fig, ax = plt.subplots(figsize=(12, 10))

cores = plt.cm.Set3(np.linspace(0, 1, len(freq_centro)))
bars = ax.barh(freq_centro['Centro/Unidade'], freq_centro['Frequência'], color=cores)
ax.set_xlabel('Número de Respondentes')
ax.set_title('Distribuição por Centro/Unidade Acadêmica')
ax.invert_yaxis()

# Adicionar labels nas barras
for bar, freq, pct in zip(bars, freq_centro['Frequência'], freq_centro['Percentual']):
    ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
            f'{freq} ({pct}%)', va='center', fontsize=8)

plt.tight_layout()
plt.savefig('grafico_centro_etapa2.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_centro_etapa2.png")

### 5.2 Agrupamento por Tipo de Unidade

In [ ]:
# Classificar unidades por tipo
def classificar_unidade(unidade):
    if pd.isna(unidade):
        return 'Não informado'
    unidade = str(unidade).upper()
    if 'CAA' in unidade or 'CAV' in unidade or 'CAS' in unidade:
        return 'Campus Fora de Sede'
    elif 'PRO' in unidade or 'REITORIA' in unidade or 'GABINETE' in unidade:
        return 'Órgãos Administrativos'
    elif 'HOSPITAL' in unidade or 'HC' in unidade:
        return 'Hospital das Clínicas'
    elif 'OUTRO' in unidade:
        return 'Outros'
    else:
        return 'Centros Acadêmicos (Recife)'

df['Tipo_Unidade'] = df['Centro_Unidade'].apply(classificar_unidade)

# Frequência por tipo de unidade
freq_tipo_unidade = df['Tipo_Unidade'].value_counts().reset_index()
freq_tipo_unidade.columns = ['Tipo de Unidade', 'Frequência']
freq_tipo_unidade['Percentual'] = (freq_tipo_unidade['Frequência'] / freq_tipo_unidade['Frequência'].sum() * 100).round(1)

print("=" * 60)
print("DISTRIBUIÇÃO POR TIPO DE UNIDADE")
print("=" * 60)
print(freq_tipo_unidade.to_string(index=False))

---

## 6. Tabela Cruzada: Vínculo × Centro/Unidade

In [ ]:
# Criar tabela cruzada
tabela_cruzada = pd.crosstab(
    df['Centro_Unidade'], 
    df['Vinculo_Padronizado'],
    margins=True,
    margins_name='Total'
)

# Ordenar por total decrescente (excluindo a linha Total)
tabela_cruzada = tabela_cruzada.sort_values('Total', ascending=False)

print("=" * 100)
print("TABELA CRUZADA: CENTRO/UNIDADE × VÍNCULO INSTITUCIONAL")
print("=" * 100)
print(tabela_cruzada)

In [ ]:
# Tabela cruzada com percentuais por linha (distribuição do vínculo dentro de cada centro)
tabela_cruzada_pct = pd.crosstab(
    df['Centro_Unidade'], 
    df['Vinculo_Padronizado'],
    normalize='index'
) * 100

tabela_cruzada_pct = tabela_cruzada_pct.round(1)

print("\n" + "=" * 100)
print("TABELA CRUZADA (% POR LINHA - Distribuição do vínculo dentro de cada centro)")
print("=" * 100)
print(tabela_cruzada_pct)

In [ ]:
# Heatmap da tabela cruzada (top 15 centros)
top_centros = freq_centro.head(15)['Centro/Unidade'].tolist()
tabela_heatmap = tabela_cruzada_pct.loc[tabela_cruzada_pct.index.isin(top_centros)]

fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(tabela_heatmap, annot=True, fmt='.1f', cmap='YlGnBu', 
            cbar_kws={'label': '% do Centro'}, ax=ax)
ax.set_title('Distribuição de Vínculos por Centro (Top 15 - % por linha)')
ax.set_xlabel('Vínculo Institucional')
ax.set_ylabel('Centro/Unidade')

plt.tight_layout()
plt.savefig('grafico_heatmap_etapa2.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_heatmap_etapa2.png")

---

## 7. Análise dos Cursos (Discentes)

In [ ]:
# Filtrar apenas discentes que informaram o curso
df_discentes = df[df['Vinculo_Padronizado'].isin(['Discente de Graduação', 'Discente de Pós-Graduação'])].copy()
df_cursos = df_discentes[df_discentes['Curso'].notna()].copy()

print(f"Total de discentes: {len(df_discentes)}")
print(f"Discentes que informaram o curso: {len(df_cursos)} ({len(df_cursos)/len(df_discentes)*100:.1f}%)")
print(f"Cursos únicos mencionados: {df_cursos['Curso'].nunique()}")

In [ ]:
# Top 20 cursos mais frequentes
freq_cursos = df_cursos['Curso'].value_counts().head(20).reset_index()
freq_cursos.columns = ['Curso', 'Frequência']
freq_cursos['Percentual'] = (freq_cursos['Frequência'] / len(df_cursos) * 100).round(1)

print("=" * 70)
print("TOP 20 CURSOS MAIS FREQUENTES")
print("=" * 70)
print(freq_cursos.to_string(index=False))

In [ ]:
# Gráfico dos top 20 cursos
fig, ax = plt.subplots(figsize=(12, 8))

cores = plt.cm.Pastel1(np.linspace(0, 1, len(freq_cursos)))
bars = ax.barh(freq_cursos['Curso'], freq_cursos['Frequência'], color=cores)
ax.set_xlabel('Número de Respondentes')
ax.set_title('Top 20 Cursos mais Frequentes (Discentes)')
ax.invert_yaxis()

for bar, freq in zip(bars, freq_cursos['Frequência']):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
            f'{freq}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('grafico_cursos_etapa2.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_cursos_etapa2.png")

---

## 8. Análise Temporal das Respostas

In [ ]:
# Converter timestamp para datetime se necessário
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extrair informações temporais
df['Data'] = df['Timestamp'].dt.date
df['Hora'] = df['Timestamp'].dt.hour
df['DiaSemana'] = df['Timestamp'].dt.day_name()

# Período de coleta
data_inicio = df['Timestamp'].min()
data_fim = df['Timestamp'].max()
duracao = (data_fim - data_inicio).days + 1

print("=" * 60)
print("PERÍODO DE COLETA")
print("=" * 60)
print(f"Data de início: {data_inicio.strftime('%d/%m/%Y %H:%M')}")
print(f"Data de término: {data_fim.strftime('%d/%m/%Y %H:%M')}")
print(f"Duração total: {duracao} dias")
print(f"Média de respostas por dia: {len(df)/duracao:.1f}")

In [ ]:
# Distribuição de respostas por dia
respostas_por_dia = df.groupby('Data').size().reset_index(name='Respostas')
respostas_por_dia['Data'] = pd.to_datetime(respostas_por_dia['Data'])

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gráfico 1: Respostas por dia
axes[0, 0].plot(respostas_por_dia['Data'], respostas_por_dia['Respostas'], 
                marker='o', linewidth=2, markersize=4, color='steelblue')
axes[0, 0].set_xlabel('Data')
axes[0, 0].set_ylabel('Número de Respostas')
axes[0, 0].set_title('Distribuição Temporal das Respostas')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].axhline(y=respostas_por_dia['Respostas'].mean(), color='red', 
                   linestyle='--', label=f'Média: {respostas_por_dia["Respostas"].mean():.1f}')
axes[0, 0].legend()

# Gráfico 2: Respostas acumuladas
respostas_por_dia['Acumulado'] = respostas_por_dia['Respostas'].cumsum()
axes[0, 1].fill_between(respostas_por_dia['Data'], respostas_por_dia['Acumulado'], 
                        alpha=0.3, color='steelblue')
axes[0, 1].plot(respostas_por_dia['Data'], respostas_por_dia['Acumulado'], 
                linewidth=2, color='steelblue')
axes[0, 1].set_xlabel('Data')
axes[0, 1].set_ylabel('Respostas Acumuladas')
axes[0, 1].set_title('Respostas Acumuladas ao Longo do Tempo')
axes[0, 1].tick_params(axis='x', rotation=45)

# Gráfico 3: Respostas por hora do dia
respostas_por_hora = df.groupby('Hora').size()
axes[1, 0].bar(respostas_por_hora.index, respostas_por_hora.values, color='steelblue', alpha=0.7)
axes[1, 0].set_xlabel('Hora do Dia')
axes[1, 0].set_ylabel('Número de Respostas')
axes[1, 0].set_title('Distribuição por Hora do Dia')
axes[1, 0].set_xticks(range(0, 24))

# Gráfico 4: Respostas por dia da semana
ordem_dias = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
nomes_dias = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo']
respostas_por_diasemana = df['DiaSemana'].value_counts().reindex(ordem_dias)
axes[1, 1].bar(nomes_dias, respostas_por_diasemana.values, color='steelblue', alpha=0.7)
axes[1, 1].set_xlabel('Dia da Semana')
axes[1, 1].set_ylabel('Número de Respostas')
axes[1, 1].set_title('Distribuição por Dia da Semana')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('grafico_temporal_etapa2.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_temporal_etapa2.png")

---

## 9. Comparação com Dados Institucionais (Representatividade)

In [ ]:
# ============================================================================
# DADOS INSTITUCIONAIS DA UFPE (aproximados - ajuste conforme dados oficiais)
# Fonte: Relatórios institucionais da UFPE (inserir dados reais quando disponíveis)
# ============================================================================

# Dados aproximados da comunidade UFPE (ajustar conforme dados oficiais)
dados_institucionais = {
    'Discente de Graduação': 30000,    # Aproximado
    'Discente de Pós-Graduação': 8000,  # Aproximado
    'Docente': 2800,                    # Aproximado
    'Técnico-administrativo': 3500,     # Aproximado
    'Egresso': None                     # Não disponível para comparação
}

# Criar tabela de comparação
comparacao = []
for vinculo, pop_total in dados_institucionais.items():
    n_amostra = len(df[df['Vinculo_Padronizado'] == vinculo])
    pct_amostra = n_amostra / len(df) * 100
    
    if pop_total:
        pct_populacao = pop_total / sum(v for v in dados_institucionais.values() if v) * 100
        taxa_resposta = n_amostra / pop_total * 100
        diferenca = pct_amostra - pct_populacao
    else:
        pct_populacao = '-'
        taxa_resposta = '-'
        diferenca = '-'
    
    comparacao.append({
        'Vínculo': vinculo,
        'N Amostra': n_amostra,
        '% Amostra': round(pct_amostra, 1),
        'Pop. Estimada': pop_total if pop_total else '-',
        '% População': round(pct_populacao, 1) if isinstance(pct_populacao, float) else pct_populacao,
        'Taxa Resposta (%)': round(taxa_resposta, 2) if isinstance(taxa_resposta, float) else taxa_resposta,
        'Diferença (pp)': round(diferenca, 1) if isinstance(diferenca, float) else diferenca
    })

df_comparacao = pd.DataFrame(comparacao)

print("=" * 100)
print("ANÁLISE DE REPRESENTATIVIDADE")
print("=" * 100)
print("\n⚠️  NOTA: Os dados populacionais são estimativas. Ajuste conforme dados oficiais da UFPE.\n")
print(df_comparacao.to_string(index=False))

print("\n" + "-" * 100)
print("INTERPRETAÇÃO:")
print("-" * 100)
print("• Diferença positiva (+): grupo SOBRERREPRESENTADO na amostra")
print("• Diferença negativa (-): grupo SUBREPRESENTADO na amostra")
print("• Taxa de resposta: proporção do grupo que respondeu à consulta")

In [ ]:
# Gráfico de comparação
df_comp_plot = df_comparacao[df_comparacao['Pop. Estimada'] != '-'].copy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Comparação Amostra vs População
x = np.arange(len(df_comp_plot))
width = 0.35

bars1 = axes[0].bar(x - width/2, df_comp_plot['% Amostra'], width, label='% Amostra', color='steelblue')
bars2 = axes[0].bar(x + width/2, df_comp_plot['% População'], width, label='% População', color='coral')

axes[0].set_xlabel('Vínculo')
axes[0].set_ylabel('Percentual (%)')
axes[0].set_title('Comparação: Amostra vs População Estimada')
axes[0].set_xticks(x)
axes[0].set_xticklabels(df_comp_plot['Vínculo'], rotation=45, ha='right')
axes[0].legend()

# Gráfico 2: Taxa de resposta por vínculo
df_comp_plot['Taxa Resposta (%)'] = pd.to_numeric(df_comp_plot['Taxa Resposta (%)'])
bars = axes[1].bar(df_comp_plot['Vínculo'], df_comp_plot['Taxa Resposta (%)'], color='seagreen')
axes[1].set_xlabel('Vínculo')
axes[1].set_ylabel('Taxa de Resposta (%)')
axes[1].set_title('Taxa de Resposta por Vínculo')
axes[1].tick_params(axis='x', rotation=45)

for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.1f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('grafico_representatividade_etapa2.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nGráfico salvo como: grafico_representatividade_etapa2.png")

---

## 10. Resumo e Estatísticas Descritivas

In [ ]:
print("=" * 70)
print("RESUMO DO PERFIL DOS RESPONDENTES")
print("=" * 70)

print(f"""
📊 VISÃO GERAL DA AMOSTRA

   Total de respondentes: {len(df)}
   Período de coleta: {data_inicio.strftime('%d/%m/%Y')} a {data_fim.strftime('%d/%m/%Y')} ({duracao} dias)
   Média de respostas/dia: {len(df)/duracao:.1f}

📌 DISTRIBUIÇÃO POR VÍNCULO
""")

for _, row in freq_vinculo.iterrows():
    print(f"   • {row['Vínculo']}: {row['Frequência']} ({row['Percentual']}%)")

print(f"""
📌 DISTRIBUIÇÃO GEOGRÁFICA

   Centros/Unidades representados: {len(freq_centro)}
   Top 3 centros:""")

for _, row in freq_centro.head(3).iterrows():
    print(f"   • {row['Centro/Unidade']}: {row['Frequência']} ({row['Percentual']}%)")

print(f"""
📌 CURSOS (DISCENTES)

   Discentes que informaram curso: {len(df_cursos)} de {len(df_discentes)} ({len(df_cursos)/len(df_discentes)*100:.1f}%)
   Cursos únicos: {df_cursos['Curso'].nunique()}

📌 PADRÃO TEMPORAL

   Dia com mais respostas: {respostas_por_dia.loc[respostas_por_dia['Respostas'].idxmax(), 'Data'].strftime('%d/%m/%Y')} ({respostas_por_dia['Respostas'].max()} respostas)
   Horário de pico: {respostas_por_hora.idxmax()}h ({respostas_por_hora.max()} respostas)
""")

---

## 11. Salvar Resultados

In [ ]:
# Salvar todas as tabelas em um arquivo Excel com múltiplas abas
with pd.ExcelWriter(ARQUIVO_SAIDA_EXCEL, engine='openpyxl') as writer:
    freq_vinculo.to_excel(writer, sheet_name='Freq_Vinculo', index=False)
    freq_centro.to_excel(writer, sheet_name='Freq_Centro', index=False)
    tabela_cruzada.to_excel(writer, sheet_name='Cruzada_Centro_Vinculo')
    tabela_cruzada_pct.to_excel(writer, sheet_name='Cruzada_Percentual')
    freq_cursos.to_excel(writer, sheet_name='Top20_Cursos', index=False)
    respostas_por_dia.to_excel(writer, sheet_name='Respostas_por_Dia', index=False)
    df_comparacao.to_excel(writer, sheet_name='Representatividade', index=False)

print(f"✅ Tabelas salvas em: {ARQUIVO_SAIDA_EXCEL}")

In [ ]:
# Combinar todos os gráficos em uma única imagem
from PIL import Image
import os

# Lista de gráficos gerados
graficos = [
    'grafico_vinculo_etapa2.png',
    'grafico_centro_etapa2.png',
    'grafico_heatmap_etapa2.png',
    'grafico_cursos_etapa2.png',
    'grafico_temporal_etapa2.png',
    'grafico_representatividade_etapa2.png'
]

print("Gráficos gerados:")
for g in graficos:
    if os.path.exists(g):
        print(f"  ✅ {g}")
    else:
        print(f"  ❌ {g} (não encontrado)")

In [ ]:
# Gerar relatório em Markdown
relatorio = f"""# RELATÓRIO DO PERFIL DOS RESPONDENTES
## Etapa 2 — Consulta à Comunidade UFPE sobre o Uso de IA

**Data de execução:** {datetime.now().strftime("%d/%m/%Y às %H:%M")}

---

## 1. VISÃO GERAL DA AMOSTRA

| Métrica | Valor |
|---------|-------|
| Total de respondentes | {len(df)} |
| Período de coleta | {data_inicio.strftime('%d/%m/%Y')} a {data_fim.strftime('%d/%m/%Y')} |
| Duração | {duracao} dias |
| Média de respostas/dia | {len(df)/duracao:.1f} |

---

## 2. DISTRIBUIÇÃO POR VÍNCULO INSTITUCIONAL

| Vínculo | Frequência | Percentual |
|---------|------------|------------|
"""

for _, row in freq_vinculo.iterrows():
    relatorio += f"| {row['Vínculo']} | {row['Frequência']} | {row['Percentual']}% |\n"

relatorio += f"""
---

## 3. DISTRIBUIÇÃO POR CENTRO/UNIDADE

Total de centros/unidades representados: **{len(freq_centro)}**

### Top 10 Centros/Unidades

| Centro/Unidade | Frequência | Percentual |
|----------------|------------|------------|
"""

for _, row in freq_centro.head(10).iterrows():
    relatorio += f"| {row['Centro/Unidade']} | {row['Frequência']} | {row['Percentual']}% |\n"

relatorio += f"""
---

## 4. ANÁLISE TEMPORAL

| Métrica | Valor |
|---------|-------|
| Dia com mais respostas | {respostas_por_dia.loc[respostas_por_dia['Respostas'].idxmax(), 'Data'].strftime('%d/%m/%Y')} ({respostas_por_dia['Respostas'].max()} respostas) |
| Horário de pico | {respostas_por_hora.idxmax()}h ({respostas_por_hora.max()} respostas) |

---

## 5. ANÁLISE DE REPRESENTATIVIDADE

⚠️ **Nota:** Os dados populacionais são estimativas. Ajuste conforme dados oficiais da UFPE.

| Vínculo | N Amostra | % Amostra | Pop. Estimada | % População | Taxa Resposta |
|---------|-----------|-----------|---------------|-------------|---------------|
"""

for _, row in df_comparacao.iterrows():
    relatorio += f"| {row['Vínculo']} | {row['N Amostra']} | {row['% Amostra']}% | {row['Pop. Estimada']} | {row['% População']}% | {row['Taxa Resposta (%)']}% |\n"

relatorio += f"""
---

## 6. PRINCIPAIS OBSERVAÇÕES

1. **Composição da amostra:** A maioria dos respondentes são discentes de graduação ({freq_vinculo.iloc[0]['Percentual']}%), seguidos por docentes ({freq_vinculo.iloc[1]['Percentual']}%).

2. **Cobertura geográfica:** Todos os {len(freq_centro)} centros/unidades da UFPE estão representados na amostra.

3. **Representatividade:** Docentes apresentam a maior taxa de resposta proporcional à população, enquanto discentes de graduação são o grupo mais numeroso em termos absolutos.

4. **Padrão temporal:** As respostas foram coletadas ao longo de {duracao} dias, com pico de atividade no horário das {respostas_por_hora.idxmax()}h.

---

## 7. ARQUIVOS GERADOS

| Arquivo | Descrição |
|---------|----------|
| `{ARQUIVO_SAIDA_EXCEL}` | Tabelas de frequência e cruzamentos |
| `grafico_vinculo_etapa2.png` | Distribuição por vínculo |
| `grafico_centro_etapa2.png` | Distribuição por centro |
| `grafico_heatmap_etapa2.png` | Heatmap vínculo × centro |
| `grafico_cursos_etapa2.png` | Top 20 cursos |
| `grafico_temporal_etapa2.png` | Análise temporal |
| `grafico_representatividade_etapa2.png` | Comparação com população |

---

*Relatório gerado automaticamente em {datetime.now().strftime("%d/%m/%Y às %H:%M")}*
"""

# Salvar relatório
with open(ARQUIVO_SAIDA_RELATORIO, "w", encoding='utf-8') as f:
    f.write(relatorio)

print(f"✅ Relatório salvo em: {ARQUIVO_SAIDA_RELATORIO}")

In [ ]:
print("\n" + "=" * 70)
print("ETAPA 2 CONCLUÍDA COM SUCESSO!")
print("=" * 70)
print(f"""
📁 ARQUIVOS GERADOS:

   📊 Dados:
      • {ARQUIVO_SAIDA_EXCEL}

   📈 Gráficos:
      • grafico_vinculo_etapa2.png
      • grafico_centro_etapa2.png
      • grafico_heatmap_etapa2.png
      • grafico_cursos_etapa2.png
      • grafico_temporal_etapa2.png
      • grafico_representatividade_etapa2.png

   📝 Relatório:
      • {ARQUIVO_SAIDA_RELATORIO}

▶️  PRÓXIMA ETAPA: Etapa 3 - Análise das Variáveis Categóricas Fechadas
""")

---

## Resumo dos Arquivos

### Entrada
| Arquivo | Origem |
|---------|--------|
| `dados_limpos_etapa1.xlsx` | Etapa 1 |

### Saída
| Arquivo | Descrição |
|---------|----------|
| `perfil_respondentes_etapa2.xlsx` | Tabelas de frequência e cruzamentos |
| `grafico_vinculo_etapa2.png` | Distribuição por vínculo institucional |
| `grafico_centro_etapa2.png` | Distribuição por centro/unidade |
| `grafico_heatmap_etapa2.png` | Heatmap vínculo × centro |
| `grafico_cursos_etapa2.png` | Top 20 cursos (discentes) |
| `grafico_temporal_etapa2.png` | Análise temporal das respostas |
| `grafico_representatividade_etapa2.png` | Comparação amostra vs população |
| `relatorio_perfil_etapa2.md` | Relatório descritivo completo |

---

**Próxima etapa:** Etapa 3 - Análise das Variáveis Categóricas Fechadas